# Задание 4.  Обучение нейронных сетей

Датасет: https://www.kaggle.com/competitions/digit-recognizer/data

Для представленного датасета необходимо:
1. Решить задачу многоклассовой классификации с помощью многослойной
нейронной сети, состоящей из полносвязных слоев.
2. Вычислить градиент целевой функции по обучаемым параметрам нейронной
сети, сравнить полученные значения со значениями, вычисленными с помощью
библиотечных функций
3. Определить параметры нейронной сети, при которых точность классификации
максимальна

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
# Загрузка данных
train_data = pd.read_csv("D:/Projects/University/ML/Task4/digit-recognizer/train.csv")
test_data = pd.read_csv("D:/Projects/University/ML/Task4/digit-recognizer/test.csv")
train_data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Разделение на признаки (X) и метки (y)
X = train_data.drop("label", axis=1).values.astype(np.float32)
y = train_data["label"].values

# Нормализация данных
# X = X / 255.0
X = np.array(X)
X = X.astype('float32')
X = X / 255.0
X = X.reshape(-1, 28, 28, 1)  # Изменение формата изображений

# Разделение на тренировочный и валидационный наборы
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### Создадим класс для нейронной сети:

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, hidden_size)  # Добавлен слой
        self.relu2 = nn.ReLU()  # Добавлен ReLU
        self.layer3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.layer2(x)  # Применение второго линейного слоя
        x = self.relu2(x)  # Применение второго ReLU
        x = self.layer3(x)
        return x

Теперь создадим модель:

In [5]:
# Определение параметров
input_size = 28 * 28 * 1  # количество пикселей в изображении
hidden_size = 512
output_size = 10  # количество классов (цифр от 0 до 9)

# Создание модели
model = NeuralNetwork(input_size, hidden_size, output_size)

Теперь определим функцию потерь:

In [6]:
criterion = nn.CrossEntropyLoss()

In [7]:
def run_model(num_epochs=10, lr=0.001, logs=False):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        inputs = Variable(torch.from_numpy(X_train.reshape(-1, 28 * 28 * 1)))
        labels = Variable(torch.from_numpy(y_train))
    
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
    
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        if logs:
            if (epoch + 1) % 1 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
    
    # Оценка модели
    model.eval()
    with torch.no_grad():
        inputs = Variable(torch.from_numpy(X_val.reshape(-1, 28 * 28 * 1)))
        labels = Variable(torch.from_numpy(y_val))
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        accuracy = (predicted == labels).sum().item() / labels.size(0)

    if logs:
        print(f'Validation Accuracy: {accuracy}')
    return accuracy

# Запускаем модель при дефолтных параметрах
run_model(logs=True)

Epoch [1/10], Loss: 2.305510997772217
Epoch [2/10], Loss: 2.2226595878601074
Epoch [3/10], Loss: 2.1289143562316895
Epoch [4/10], Loss: 2.003002405166626
Epoch [5/10], Loss: 1.8416163921356201
Epoch [6/10], Loss: 1.6539793014526367
Epoch [7/10], Loss: 1.4530905485153198
Epoch [8/10], Loss: 1.2501496076583862
Epoch [9/10], Loss: 1.0614486932754517
Epoch [10/10], Loss: 0.8986860513687134
Validation Accuracy: 0.8253571428571429


0.8253571428571429

### Определить параметры нейронной сети, при которых точность классификации максимальна

Попробуем перебрать несколько сочетаний параметров learning rate и Epoch.

In [8]:
lr_values = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
epochs_values = [5, 10, 15]

In [9]:
def grid_search(learning_rates, epochs_list):
    best_lr = None
    best_epochs = None
    best_accuracy = 0.0

    for lr in learning_rates:
        for epochs in epochs_list:
            print(f"Training with lr={lr}, epochs={epochs}")
            accuracy = run_model(num_epochs=epochs, lr=lr, logs=False)
            print(f"Total Accuracy for combination: {accuracy}")
            print("---------------------------------------------------")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_lr = lr
                best_epochs = epochs

    print(f"Best model achieved with lr={best_lr}, epochs={best_epochs}, accuracy={best_accuracy}")
    return best_lr, best_epochs

In [10]:
best_lr, best_epochs = grid_search(lr_values, epochs_values)

Training with lr=0.001, epochs=5
Total Accuracy for combination: 0.8608333333333333
---------------------------------------------------
Training with lr=0.001, epochs=10
Total Accuracy for combination: 0.8798809523809524
---------------------------------------------------
Training with lr=0.001, epochs=15
Total Accuracy for combination: 0.9217857142857143
---------------------------------------------------
Training with lr=0.005, epochs=5
Total Accuracy for combination: 0.606547619047619
---------------------------------------------------
Training with lr=0.005, epochs=10
Total Accuracy for combination: 0.9021428571428571
---------------------------------------------------
Training with lr=0.005, epochs=15
Total Accuracy for combination: 0.9411904761904762
---------------------------------------------------
Training with lr=0.01, epochs=5
Total Accuracy for combination: 0.6711904761904762
---------------------------------------------------
Training with lr=0.01, epochs=10
Total Accurac

Таким образом лучший показатель accuracy (95.5%) модель показывает при **learning rate = 0.01 и Epochs = 15**.